In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, recall_score, f1_score, precision_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from collections import Counter

In [ ]:
tablet = pd.read_csv("../input/tabletpc-priceclassification/tablet.csv")
df=tablet.copy()

In [ ]:
#Veriyi tanıma,sindirme
#Veri dengelimi inceleme
#Korelasyon Matrisi

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.corr()

In [ ]:
df.describe().T

In [ ]:
#Veri Görselleştirmeleri

In [ ]:
#ısı haritasında Arka Kamere ve Ön kamare arasında güçlü bir ilişki var
#CozunurlukYükseklik ve CozunurlukGenislik arasında güçlü bir ilişki var

In [ ]:
plt.subplots(figsize=(10, 10))
sns.heatmap(df.corr(), annot=True,cmap="YlGnBu")
plt.show()

In [ ]:
df.hist(figsize = (15,15),color="purple")

plt.show()
#Tabletlerin ağırlıklarının,batarya gücünün,Cozunurluk Genisliklerinin,DahiliBelleklerin ve Ramlerin genel olarak dengeli dağıldığını görüyoruz.

In [ ]:
sns.pairplot(df);

In [ ]:
def countplot(baslik): 
    sns.countplot(x=baslik, data=df)
    plt.xticks(rotation=50)
    plt.show()
    
    print(df[baslik].value_counts())

In [ ]:
countplot("FiyatAraligi")

In [ ]:
countplot("Renk")

In [ ]:
countplot("Bluetooth")

In [ ]:
countplot("CiftHat")

In [ ]:
countplot("4G")

In [ ]:
countplot("3G")

In [ ]:
countplot("Dokunmatik")

In [ ]:
countplot("WiFi")

In [ ]:
#Yukarıdaki grafiklerde gördüğümüz gibi sayısal olmayan verilerde 3G teknolojisi hariç stabil bir dağılım söz konusu

In [ ]:
#Ortalama,Medyan ve Standart sapma

In [ ]:
df.mean()#Verilerin ortalamalı ile medyanlarının çok yakın olduğunu görüyoruz yani veriler gayet stabil demektir.

In [ ]:
df.std()
#Verilerin değerlerinin yakınlığını inceliyoruz.Standar sapmaları küçük olanların verileri birbirlerine daha yakın değerler içermektedir.

In [ ]:
df.median()

In [ ]:
#Ön işleme kısmı
#Eksik verlerin doldurulması
#Sayısallaşrılıma işlemleri

In [ ]:
df.isna().sum()
#5 tane 4G,12 tane Ram verilerinde eksik var. Bu eksikler hesaplarımızda yanılmamıza sebep olabilir.Bir sonraki işlemde eksik verileri dolduracağız.

In [ ]:
df.RAM = df.RAM.fillna(df.RAM.mean())#Eksik verileri ortalamaları ile dolduruyoruz.Çünkü verilerde sapmaya sebep olmamasını istiyoruz.

In [ ]:
df.OnKameraMP = df.OnKameraMP.fillna(df.OnKameraMP.mean())

In [ ]:
df.Bluetooth = df.Bluetooth.eq('Var').mul(1)
df.CiftHat = df.CiftHat.eq('Var').mul(1)
df["4G"] = df["4G"].eq('Var').mul(1)
df["3G"] = df["3G"].eq('Var').mul(1)
df.Dokunmatik = df.Dokunmatik.eq('Var').mul(1)
df.WiFi = df.WiFi.eq('Var').mul(1)

In [ ]:
df.FiyatAraligi=pd.Categorical(df["FiyatAraligi"],ordered=True,categories=["Çok Ucuz","Ucuz","Normal","Pahalı"])

In [ ]:
onehot_encoder = OneHotEncoder()

In [ ]:
df.Renk = onehot_encoder.fit_transform(df[["Renk"]]).toarray()

In [ ]:
df.Renk

In [ ]:
#Makine Öğrenmesi

In [ ]:
X = df.drop("FiyatAraligi",axis=1)
y= df["FiyatAraligi"]

In [ ]:
X

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [ ]:
X_train

In [ ]:
X_test

In [ ]:
y_train

In [ ]:
y_test

In [ ]:
#LogisticRegression

In [ ]:
logistic_regression=LogisticRegression()

In [ ]:
logistic_model=logistic_regression.fit(X_train,y_train)

In [ ]:
y_pred_LR=logistic_model.predict(X_test)

In [ ]:
df_logistic = pd.DataFrame({"Gerçek Değerler" : y_test, "Tahmin Edilen" : y_pred_LR,"Tahmin Sonucu":(y_test==y_pred_LR)})

df_logistic

In [ ]:
print("R Squared:", logistic_model.score(X_test, y_test))
print(classification_report(y_test, y_pred_LR))

In [ ]:
LR_karmasiklik_matrisi = confusion_matrix(y_test, y_pred_LR)
print(LR_karmasiklik_matrisi)

In [ ]:
#GaussianNB

In [ ]:
Gaussian_NB= GaussianNB()

In [ ]:
NB_model = Gaussian_NB.fit(X_train,y_train)

In [ ]:
y_pred_NB=NB_model.predict(X_test)

In [ ]:
df_NB = pd.DataFrame({"Gerçek Değerler" : y_test, "Tahmin Edilen" : y_pred_NB,"Tahmin Sonucu":(y_test==y_pred_NB)})

df_NB

In [ ]:
print("R Squared:", NB_model.score(X_test, y_test))
print(classification_report(y_test, y_pred_NB))

In [ ]:
DT_karmasiklik_matrisi = confusion_matrix(y_test, y_pred_NB)
print(DT_karmasiklik_matrisi)

In [ ]:
#DecisionTree

In [ ]:
DecisionTree = tree.DecisionTreeClassifier()

In [ ]:
DecisionTree_model=DecisionTree.fit(X_train,y_train)

In [ ]:
y_pred_DT=DecisionTree_model.predict(X_test)

In [ ]:
df_DecisionTree = pd.DataFrame({"Gerçek Değerler" : y_test, "Tahmin Edilen" : y_pred_DT,"Tahmin Sonucu":(y_test==y_pred_DT)})

df_DecisionTree

In [ ]:
print("R Squared:", DecisionTree_model.score(X_test, y_test))
print(classification_report(y_test, y_pred_DT))

In [ ]:
DT_karmasiklik_matrisi = confusion_matrix(y_test, y_pred_DT)
print(DT_karmasiklik_matrisi)

In [ ]:
#DecisionTree=>criterion='entropy'

In [ ]:
DecisionTree_entropy = tree.DecisionTreeClassifier(criterion='entropy')

In [ ]:
DecisionTree_entropy_model=DecisionTree_entropy.fit(X_train,y_train)

In [ ]:
y_pred_DT_entropy=DecisionTree_entropy_model.predict(X_test)

In [ ]:
df_DecisionTree_entropy = pd.DataFrame({"Gerçek Değerler" : y_test, "Tahmin Edilen" : y_pred_DT,"Tahmin Sonucu":(y_test==y_pred_DT_entropy)})

df_DecisionTree_entropy

In [ ]:
print("R Squared:", DecisionTree_entropy_model.score(X_test, y_test))
print(classification_report(y_test, y_pred_DT_entropy))

In [ ]:
DT_karmasiklik_matrisi_entropy = confusion_matrix(y_test, y_pred_DT_entropy)
print(DT_karmasiklik_matrisi_entropy)

In [ ]:
#KNN

In [ ]:
KNN = KNeighborsClassifier()

In [ ]:
KNN_model=KNN.fit(X_train,y_train)

In [ ]:
y_pred_KNN=KNN_model.predict(X_test)

In [ ]:
df_KNN = pd.DataFrame({"Gerçek Değerler" : y_test, "Tahmin Edilen" : y_pred_KNN,"Tahmin Sonucu":(y_test==y_pred_KNN)})

df_KNN

In [ ]:
print("R Squared:", KNN_model.score(X_test, y_test))
print(classification_report(y_test, y_pred_KNN))

In [ ]:
KNN_karmasiklik_matrisi = confusion_matrix(y_test, y_pred_KNN)
print(KNN_karmasiklik_matrisi)

In [ ]:
#KNN=>2,15

In [ ]:
array = []
for i in range (2,15):
    KNN_n_neighbors = KNeighborsClassifier(n_neighbors=i)
    KNN_model_n_neighbors=KNN_n_neighbors.fit(X_train,y_train)
    y_pred_KNN_n_neighbors=KNN_model_n_neighbors.predict(X_test)
    print(i,".R Squared:", KNN_model_n_neighbors.score(X_test, y_test))
    KNN_score = KNN_model_n_neighbors.score(X_test, y_test)
    array.insert(i,KNN_score)

In [ ]:
plt.plot(array)
plt.xlabel("k değerleri")
plt.ylabel("doğruluk skoru")

In [ ]:
#KNN modeli 0.934 ile en yüksek başarı puanını veriyor.(n_neighbors=7 iken) daha sonra DT modeli 0.838 değeri geliyor ve en son GaussianNB modeli gelmekte